# Data Processing - PART 1

## Clean Data

Notebook that cleans all my data and outputs to final_data.csv<br>
Note in my data collection I collected car_list.csv (grabbed links) and then car_scrap.csv (page scraped). Here i will have to merge these two.<br><br>
Additionally I performed the data collection twice once for cars around seattle and once for cars around portland.
Therefore I rename my files Seattle -> seattle_250.csv (contains grabbed links) seattle_car_scrap.csv (contains the page scrap for each of the stocknos in seattle_250.csv<br><br>

Similarly for portland.
I used this structure so that if need I can expand my database one city at a time. 

In [3]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns

# allow render in Jupyter notebook
%matplotlib inline
%pylab inline 
%config IPCompleter.greedy=True
%config InlineBackend.figure_format = 'svg'
sns.set_style("white")

Populating the interactive namespace from numpy and matplotlib


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 30)

### 1. Reading and looking at my data

In [3]:
#read files from car list
car_list=pd.read_csv('../data/port_250.csv')
car_list.drop('Unnamed: 0',axis =1, inplace = True)
car_list.head()

,stock_n,year,brand,model,price
0,16338689,2012,Honda,Accord EX-L,14998.0
1,16005669,2017,Honda,Accord LX-S,20998.0
2,16238292,2015,Honda,Accord Sport,16998.0
3,16238355,2017,Honda,Accord Sport SE,22998.0
4,16238739,2017,Honda,Accord Hybrid,22998.0


Labelling for car brands makes sense for 
Ford             174
Toyota           155
Chevrolet        114
Nissan           101
Dodge             72
Hyundai           71
Jeep              65
Honda             64
BMW               49
covers 70% of the market 
maybe kia, volkswagen, lexus ... beyond others

In [4]:
# read scrapped details
car_details=pd.read_csv('../data/port_car_scrap.csv')
car_details.drop('Unnamed: 0',axis =1, inplace = True)
car_details.head()

,stock_n,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,torque,comp_ratio,cam_shaft,engine_type,bore,stroke,values_per_cylinder,fuel_cap,epa_mpg,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,tow_cap,pay_cap,gross,tire,trans
0,16338689,73K,Black,Black,2WDWD,3.5L engine,6 cylinders,271 horsepower@6000rpm,254 torque@5000rpm,11:1,Single Overhead Cam,Gas,3.50,3.66,4.0,NaN,17/26,"107.9""","191.0""",N/R,56,3410 lbs.,"42.2""","37.4""",5/5,12 cu. ft.,1000 lbs.,850 lbs.,4475 lbs.,NaN,5-Speed Automatic
1,16005669,5K,Black,Blue,2WDWD,2.4L engine,4 cylinders,185 horsepower@6400rpm,NaN,11.10,Variable Timing,Gas,3.43,3.90,4.0,NaN,27/36,107.3'',189.5'',73.0'',56.5'',3248 lbs.,42.2'',39.0'',5/5,13.7 cu. ft.,NaN,NaN,NaN,NaN,NaN
2,16238292,40K,Black,Red,2WDWD,2.4L engine,4 cylinders,189 horsepower@6400rpm,182 torque@3900rpm,11:1,Variable Timing DOHC,Gas,3.43,3.90,4.0,NaN,27/36,"109.3""","191.4""",72,57,3342 lbs.,"42.5""","39.1""",5/5,16 cu. ft.,NaN,NaN,N/R,NaN,NaN
3,16238355,21K,Black,Black,2WDWD,2.4L engine,4 cylinders,189 horsepower@6400rpm,NaN,11.10,Variable Timing,Gas,3.43,3.90,4.0,NaN,27/36,109.3'',192.5'',72.8'',57.7'',3369 lbs.,42.5'',39.1'',5/5,15.8 cu. ft.,NaN,NaN,NaN,NaN,NaN
4,16238739,23K,Tan,Blue,2WDWD,2.0L engine,4 cylinders,212 horsepower@6200rpm,NaN,13.00,Variable Timing,Gas/Electric,3.19,3.81,4.0,NaN,49/47,109.3'',194.1'',72.8'',57.5'',3550 lbs.,42.5'',39.5'',5/5,13.5 cu. ft.,NaN,NaN,NaN,NaN,NaN


In [5]:
print(car_details.shape)
print(car_details.info())

(1030, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1030 entries, 0 to 1029
Data columns (total 31 columns):
stock_n                1030 non-null int64
mileage                1030 non-null object
interior_color         1020 non-null object
exterior_color         1030 non-null object
drive_type             1029 non-null object
engine_size            1030 non-null object
num_cylinders          1018 non-null object
hp                     1018 non-null object
torque                 797 non-null object
comp_ratio             999 non-null object
cam_shaft              992 non-null object
engine_type            1018 non-null object
bore                   1016 non-null float64
stroke                 1016 non-null float64
values_per_cylinder    1018 non-null float64
fuel_cap               48 non-null float64
epa_mpg                991 non-null object
wheel_base             1015 non-null object
length                 1015 non-null object
width                  1015 non-null object
heigh

### 2. Remove null entries

In [6]:
#remove columns that have high number of null
#'torque',fuel_cap, tow_cap, 'tow_cap','pay_cap','gross','tire','trans'

heading_keep =['stock_n','mileage','interior_color','exterior_color',\
          'drive_type','engine_size','num_cylinders','hp',\
          'comp_ratio','cam_shaft','engine_type','bore',\
          'stroke','values_per_cylinder','epa_mpg','wheel_base',\
          'length','width','height','curb_weight','leg_room',\
          'head_room','seating_cap','cargo_cap']
car_details= car_details[heading_keep]

In [7]:
# drop null entry rows
car_details=car_details.dropna()
print(car_details.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 925 entries, 0 to 1029
Data columns (total 24 columns):
stock_n                925 non-null int64
mileage                925 non-null object
interior_color         925 non-null object
exterior_color         925 non-null object
drive_type             925 non-null object
engine_size            925 non-null object
num_cylinders          925 non-null object
hp                     925 non-null object
comp_ratio             925 non-null object
cam_shaft              925 non-null object
engine_type            925 non-null object
bore                   925 non-null float64
stroke                 925 non-null float64
values_per_cylinder    925 non-null float64
epa_mpg                925 non-null object
wheel_base             925 non-null object
length                 925 non-null object
width                  925 non-null object
height                 925 non-null object
curb_weight            925 non-null object
leg_room               925 non-n

In [8]:
car_details.head()

,stock_n,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,comp_ratio,cam_shaft,engine_type,bore,stroke,values_per_cylinder,epa_mpg,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap
0,16338689,73K,Black,Black,2WDWD,3.5L engine,6 cylinders,271 horsepower@6000rpm,11:1,Single Overhead Cam,Gas,3.50,3.66,4.0,17/26,"107.9""","191.0""",N/R,56,3410 lbs.,"42.2""","37.4""",5/5,12 cu. ft.
1,16005669,5K,Black,Blue,2WDWD,2.4L engine,4 cylinders,185 horsepower@6400rpm,11.10,Variable Timing,Gas,3.43,3.90,4.0,27/36,107.3'',189.5'',73.0'',56.5'',3248 lbs.,42.2'',39.0'',5/5,13.7 cu. ft.
2,16238292,40K,Black,Red,2WDWD,2.4L engine,4 cylinders,189 horsepower@6400rpm,11:1,Variable Timing DOHC,Gas,3.43,3.90,4.0,27/36,"109.3""","191.4""",72,57,3342 lbs.,"42.5""","39.1""",5/5,16 cu. ft.
3,16238355,21K,Black,Black,2WDWD,2.4L engine,4 cylinders,189 horsepower@6400rpm,11.10,Variable Timing,Gas,3.43,3.90,4.0,27/36,109.3'',192.5'',72.8'',57.7'',3369 lbs.,42.5'',39.1'',5/5,15.8 cu. ft.
4,16238739,23K,Tan,Blue,2WDWD,2.0L engine,4 cylinders,212 horsepower@6200rpm,13.00,Variable Timing,Gas/Electric,3.19,3.81,4.0,49/47,109.3'',194.1'',72.8'',57.5'',3550 lbs.,42.5'',39.5'',5/5,13.5 cu. ft.


### 3. Clean each column

In [9]:
# clean mileage
car_details['mileage']=car_details['mileage'].str.replace('K','000').astype('int')

In [10]:
# clean cargo_cap
car_details['cargo_cap']=car_details['cargo_cap'].str.replace('cu.',"")
car_details['cargo_cap']=car_details['cargo_cap'].str.replace('ft.',"")
car_details.loc[car_details['cargo_cap']=='N/R','cargo_cap']=np.nan
car_details['cargo_cap']=car_details['cargo_cap'].str.strip().astype('float')
#print(car_details['cargo_cap'].value_counts())

In [11]:
#clean comp_ratio
car_details['comp_ratio'] = car_details['comp_ratio'].str.strip()

In [12]:
# clean epa_mpg city/highway split it out
car_details['city_mpg'] = car_details['epa_mpg'].str.split('/',n=1,expand=True)[0]
car_details['hw_mpg'] = car_details['epa_mpg'].str.split('/',n=1,expand=True)[1]
car_details['hw_mpg'] = car_details['hw_mpg'].astype('int')
car_details['city_mpg'] = car_details['city_mpg'].astype('int')
car_details = car_details.drop(columns=['epa_mpg'])
#car_details.head()

In [13]:
# clean interior and exterior color
car_details['interior_color']=car_details['interior_color'].str.strip()
print(car_details['interior_color'].unique())
car_details['exterior_color']=car_details['exterior_color'].str.strip()
print(car_details['exterior_color'].unique())

['Black' 'Tan' 'Gray' 'Brown' 'Cream' 'Red' 'Burgundy']
['Black' 'Blue' 'Red' 'Silver' 'Gray' 'White' 'Burgundy' 'Green' 'Gold'
 'Brown' 'Purple' 'Maroon' 'Orange' 'Tan' 'Yellow']


In [14]:
# looking at drive type
car_details['drive_type']=car_details['drive_type'].str.strip()
print(car_details['drive_type'].unique())
print(car_details['drive_type'].value_counts())

['2WDWD' '4WDWD' '2WD']
2WDWD    625
4WDWD    293
2WD        7
Name: drive_type, dtype: int64


In [15]:
# replace 2WD... with just 2 or 4
car_details['drive_type']=car_details['drive_type'].str.replace('WD','').astype('int')
#car_details.head()

In [16]:
# engine_size
car_details['engine_size']=car_details['engine_size'].str.strip()

#print(car_details['engine_size'].unique())
#print(car_details['engine_size'].value_counts())
car_details['engine_size'] = car_details['engine_size'].str.replace('engine','')
car_details['engine_size'] = car_details['engine_size'].str.replace('L','')
car_details.loc[car_details['engine_size']=='N/A','engine_size']=np.nan
car_details['engine_size'] = car_details['engine_size'].str.strip().astype('float')

In [17]:
# engine_size
car_details['num_cylinders'] = car_details['num_cylinders'].str.replace('cylinders','')
car_details['num_cylinders']=car_details['num_cylinders'].str.strip().astype('int')
print(car_details['num_cylinders'].unique())


[6 4 8 3 5]


In [18]:
# hp 
car_details['hp']=car_details['hp'].str.strip()

car_details['hp'] = car_details['hp'].str.split(' ',n=1,expand=True)

car_details['hp'] = car_details['hp'].str.strip().astype('int')
print(car_details['hp'].unique())

[271 185 189 212 196 140 174 130 117 248 138 148 145 147 198 201 303 306
 328 265 184 172 290 160 285 420 192 164 181 268 301 204 383 270 245 325
 365 380 167 153 302 210  74 332 182 261 260 170 109 179 158 250 152 224
 175 178 132  98 134  99 176 266 278 200 240 300 273 279 205 220 252 255
 180 202 310 305 169  84 355 281 149 173 283 292 287 295 370 101 188 435
 288 143 280 311 190 235 275 146 228 390 284 317  94 128 256 236 115 211
 230 308 304 335 372 360 120 315 141 177 159 276 187 161 155 241 208 118
  78 168 122 381 106 272 450 320 151 195 395 385 166 137 330 400 340   0
 291 197 259 231 121 555 323 165 144 227]


In [19]:
# cam_shaft
car_details['cam_shaft']=car_details['cam_shaft'].str.strip()
print(car_details['cam_shaft'].unique())

['Single Overhead Cam' 'Variable Timing' 'Variable Timing DOHC'
 'Dual Overhead Cam' 'Variable Timing OHC' 'DOHC' 'OHC' 'Overhead Valve']


In [20]:
# engine_type
car_details['engine_type']=car_details['engine_type'].str.strip()
print(car_details['engine_type'].unique())
print(car_details['engine_type'].value_counts())

['Gas' 'Gas/Electric' 'Turbo Gas' 'Ethanol/Gas' 'Turbo Diesel'
 'Supercharged']
Gas             664
Turbo Gas       165
Ethanol/Gas      52
Gas/Electric     36
Turbo Diesel      5
Supercharged      3
Name: engine_type, dtype: int64


In [21]:
#bore,stroke,values per cylinder
print(car_details['bore'].value_counts())
print(car_details['stroke'].value_counts())
print(car_details['values_per_cylinder'].value_counts())

# values_per... not indicative as almost all are 4 could drop 

3.46    90
3.70    85
3.50    75
3.78    72
3.54    51
3.19    42
3.17    40
3.76    40
3.25    37
3.44    28
3.43    25
3.45    23
3.39    23
3.03    22
3.64    21
        ..
3.86     4
3.30     3
2.89     2
3.10     2
3.20     2
3.16     2
3.42     2
3.58     2
3.95     2
4.07     1
3.38     1
3.24     1
3.75     1
4.21     1
3.35     1
Name: bore, Length: 53, dtype: int64
3.27    134
3.82     77
3.86     54
3.94     53
3.39     39
3.62     39
3.48     38
3.65     32
3.20     32
3.41     31
3.90     27
3.36     26
3.55     24
3.43     24
3.31     19
       ... 
4.13      2
3.28      1
3.10      1
3.80      1
3.45      1
3.60      1
2.72      1
3.72      1
4.88      1
3.93      1
3.13      1
3.33      1
3.53      1
3.17      1
3.46      1
Name: stroke, Length: 67, dtype: int64
4.0    871
2.0     41
0.0     13
Name: values_per_cylinder, dtype: int64


In [22]:
# wheelbase , length, width, height, leg_room,head_room
car_details['wheel_base']=car_details['wheel_base'].str.replace('\"',"")
car_details['wheel_base']=car_details['wheel_base'].str.replace('\'',"")
car_details['wheel_base']=car_details['wheel_base'].str.strip().astype('float')

car_details['width']=car_details['width'].str.replace('\"',"")
car_details['width']=car_details['width'].str.replace('\'',"")
car_details.loc[car_details['width']=='N/R','width']=np.nan
car_details['width']=car_details['width'].str.strip().astype('float')

car_details['length']=car_details['length'].str.replace('\"',"")
car_details['length']=car_details['length'].str.replace('\'',"")
car_details.loc[car_details['length']=='N/R','length']=np.nan
car_details['length']=car_details['length'].str.strip().astype('float')

car_details['height']=car_details['height'].str.replace('\"',"")
car_details['height']=car_details['height'].str.replace('\'',"")
car_details.loc[car_details['height']=='N/R','height']=np.nan
car_details['height']=car_details['height'].str.strip().astype('float')

car_details['leg_room']=car_details['leg_room'].str.replace('\"',"")
car_details['leg_room']=car_details['leg_room'].str.replace('\'',"")
car_details.loc[car_details['leg_room']=='N/R','leg_room']=np.nan
car_details['leg_room']=car_details['leg_room'].str.strip().astype('float')

car_details['head_room']=car_details['head_room'].str.replace('\"',"")
car_details['head_room']=car_details['head_room'].str.replace('\'',"")
car_details.loc[car_details['head_room']=='N/R','head_room']=np.nan
car_details['head_room']=car_details['head_room'].str.strip().astype('float')


# print(car_details['wheel_base'].unique())
# print(car_details['head_room'].value_counts())

In [23]:
#curb_weight

car_details['curb_weight']=car_details['curb_weight'].str.replace('lbs.',"")
car_details['curb_weight']=car_details['curb_weight'].str.strip().astype('int')
#print(car_details['curb_weight'].value_counts())

In [24]:
# take max capacity as car cap convert x/y to max(x,y)

# print(car_details['seating_cap'].value_counts())

car_details['seating_cap_a'] = car_details['seating_cap'].str.split('/',n=1,expand=True)[0]
car_details['seating_cap_b'] = car_details['seating_cap'].str.split('/',n=1,expand=True)[1]
car_details['seating_cap'] = car_details[["seating_cap_a", "seating_cap_b"]].max(axis=1)
car_details['seating_cap']=car_details['seating_cap'].astype('int')
car_details = car_details.drop(columns=['seating_cap_a', 'seating_cap_b'])

In [25]:
car_details.head()

,stock_n,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,comp_ratio,cam_shaft,engine_type,bore,stroke,values_per_cylinder,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,city_mpg,hw_mpg
0,16338689,73000,Black,Black,2,3.5,6,271,11:1,Single Overhead Cam,Gas,3.50,3.66,4.0,107.9,191.0,NaN,56.0,3410,42.2,37.4,5,12.0,17,26
1,16005669,5000,Black,Blue,2,2.4,4,185,11.10,Variable Timing,Gas,3.43,3.90,4.0,107.3,189.5,73.0,56.5,3248,42.2,39.0,5,13.7,27,36
2,16238292,40000,Black,Red,2,2.4,4,189,11:1,Variable Timing DOHC,Gas,3.43,3.90,4.0,109.3,191.4,72.0,57.0,3342,42.5,39.1,5,16.0,27,36
3,16238355,21000,Black,Black,2,2.4,4,189,11.10,Variable Timing,Gas,3.43,3.90,4.0,109.3,192.5,72.8,57.7,3369,42.5,39.1,5,15.8,27,36
4,16238739,23000,Tan,Blue,2,2.0,4,212,13.00,Variable Timing,Gas/Electric,3.19,3.81,4.0,109.3,194.1,72.8,57.5,3550,42.5,39.5,5,13.5,49,47


In [26]:
car_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 925 entries, 0 to 1029
Data columns (total 25 columns):
stock_n                925 non-null int64
mileage                925 non-null int64
interior_color         925 non-null object
exterior_color         925 non-null object
drive_type             925 non-null int64
engine_size            925 non-null float64
num_cylinders          925 non-null int64
hp                     925 non-null int64
comp_ratio             925 non-null object
cam_shaft              925 non-null object
engine_type            925 non-null object
bore                   925 non-null float64
stroke                 925 non-null float64
values_per_cylinder    925 non-null float64
wheel_base             925 non-null float64
length                 925 non-null float64
width                  854 non-null float64
height                 924 non-null float64
curb_weight            925 non-null int64
leg_room               918 non-null float64
head_room              918 non-

### 4. Join car_details with car_list

In [27]:
# make sure each row corresponds to 1 stock_no

print(car_list['stock_n'].nunique())
print(car_list.shape)

print(car_details['stock_n'].nunique())
print(car_details.shape)
# they don't

1037
(1037, 5)
924
(925, 25)


In [28]:
# looking at the extra values for car_details
extra = car_details['stock_n'].value_counts().sort_values(ascending=False)
extra = extra[extra >1]
extra

16450950    2
Name: stock_n, dtype: int64

In [29]:
# if i drop all duplicates my shape is the same the nunique for stock no which mean now each row is a unique stock no.
car_details = car_details.drop_duplicates()
car_details.shape

(924, 25)

In [30]:
# dropping similarly for car_list
car_list = car_list.drop_duplicates()
car_list.shape

# Hmm  i am not getting the same numner as nunique (1116 vs 1128). what is going on 


(1037, 5)

In [31]:
# lets check
check = car_list['stock_n'].value_counts().sort_values(ascending=False)
check = check[check > 1]
check

Series([], Name: stock_n, dtype: int64)

In [32]:
# drop stocknumber with ambigous prices
l= list(check.index.astype('int'))
car_list = car_list.loc[np.logical_not(car_list['stock_n'].isin(l)),:]
car_list.shape

(1037, 5)

both car_list and car_details have unique stock number now we do a join so that only rows where the same stock number is carried forward

In [33]:
result = pd.merge(car_list, car_details, how='inner', on=['stock_n', 'stock_n'])
result.head()

,stock_n,year,brand,model,price,mileage,interior_color,exterior_color,drive_type,engine_size,num_cylinders,hp,comp_ratio,cam_shaft,engine_type,bore,stroke,values_per_cylinder,wheel_base,length,width,height,curb_weight,leg_room,head_room,seating_cap,cargo_cap,city_mpg,hw_mpg
0,16338689,2012,Honda,Accord EX-L,14998.0,73000,Black,Black,2,3.5,6,271,11:1,Single Overhead Cam,Gas,3.50,3.66,4.0,107.9,191.0,NaN,56.0,3410,42.2,37.4,5,12.0,17,26
1,16005669,2017,Honda,Accord LX-S,20998.0,5000,Black,Blue,2,2.4,4,185,11.10,Variable Timing,Gas,3.43,3.90,4.0,107.3,189.5,73.0,56.5,3248,42.2,39.0,5,13.7,27,36
2,16238292,2015,Honda,Accord Sport,16998.0,40000,Black,Red,2,2.4,4,189,11:1,Variable Timing DOHC,Gas,3.43,3.90,4.0,109.3,191.4,72.0,57.0,3342,42.5,39.1,5,16.0,27,36
3,16238355,2017,Honda,Accord Sport SE,22998.0,21000,Black,Black,2,2.4,4,189,11.10,Variable Timing,Gas,3.43,3.90,4.0,109.3,192.5,72.8,57.7,3369,42.5,39.1,5,15.8,27,36
4,16238739,2017,Honda,Accord Hybrid,22998.0,23000,Tan,Blue,2,2.0,4,212,13.00,Variable Timing,Gas/Electric,3.19,3.81,4.0,109.3,194.1,72.8,57.5,3550,42.5,39.5,5,13.5,49,47


### 5. Write to file

In [34]:
file_name = '../data/final_data.csv'
from pathlib import Path
config = Path('../data/final_data.csv')
if config.is_file():
    print('exists = yes')
    with open(file_name, 'a') as f:
        result.to_csv(f, header=False)
else:
    print('exists = no')
    with open(file_name, 'a') as f:
        result.to_csv(f, header=True)
    

exists = yes


### 6. Check the final_data.csv
Make sure there are no duplicates as we page scrap mulitple times and keep adding these in

In [4]:
df_check = pd.read_csv('../data/final_data.csv')
df_check = df_check.drop(columns="Unnamed: 0")
df_check.shape

(1781, 29)

In [39]:
df_check['stock_n'].nunique()

1781

In [40]:
df_check = df_check.drop_duplicates()
df_check.shape

(1781, 29)

In [41]:
x=df_check['stock_n'].value_counts()
x=x[x>1]
x

Series([], Name: stock_n, dtype: int64)

In [42]:
y= list(x.index.astype('int'))
car_c = df_check
car_c = car_c.loc[np.logical_not(car_c['stock_n'].isin(y)),:]
car_c.shape

(1781, 29)

In [43]:
car_c.to_csv('../data/final_data.csv')

### next -> encode_data.ipynb